# Automatic Attendance Tracking System

## Barcode Detetction

The below code detects the bar code from an image and decodes it to get the data from barcode and returns to the caller function.It draws a rectangular box around the detected barcode and displays the data on screen too. (basically, the data is the roll number)

In [2]:
import cv2
import numpy as np
from pyzbar.pyzbar import decode

def decodeTheBarcode(img):    
    myDataList = [
        "111114"
    ]

    for barcode in decode(img):
        myData = barcode.data.decode("utf-8")
        print(myData)

        if myData in myDataList:
            color = (0, 255, 0)
        else:
            color = (0, 0, 255)
        # (255, 0, 255)

        pts = np.array([barcode.polygon], np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(img, [pts], True, color, 5)
        pts2 = barcode.rect
        cv2.putText(img, myData, (pts2[0], pts2[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        return myData


    cv2.imshow("img",img)
    cv2.waitKey(1)

## We need to store the student details in a data structure such that we can access name with roll no.

In [3]:
#Keys are roll numbers and Value are Names
StudDetails ={'19501A0552':'farooq' ,'101':'alluarjun' ,'102':'mahesh','103':'charan','104':'ntr' ,'105':'prabhas' }

## Writing data in the excel sheet

I created a new object for worksheet and sent to the below function along with the primary details like roll number and the row in which it is to be added.
The name of the excel file would be 'Attendance of' followed by the date of taking attendance 
Creating multiple wroksheet objects with same file name creates a new file each time and finally saves the last file.
So, I used Single object every time

In [4]:
def Write(rolno , row , xlsheet):
    xlsheet.write(row,0,row)
    xlsheet.write(row,1,rolno)
    xlsheet.write(row,2,StudDetails[rolno])
    xlsheet.write(row,3,str(date.today()))
    xlsheet.write(row,4,str(datetime.now().time()))    

## Verification of the roll number and face.

It first detects the roll number if it is present in the StudDetails then it will Search for the matching face.

It displays no face detected if it unable to detect the face ( haarcascade_frontalface_default is used for face detection )

If the person infront of the camera doesn't match with the student with roll number, it displays "You are not StudentX" 

Once after matching the roll number and face It shows "Registered" and enter his details in the Excel Sheet

It stops the process when any key is pressed or once it is shown barcode with 'Stop' as data

In [8]:
import base64
from deepface import DeepFace
import cv2
import numpy as np
import time
from PIL import Image
import xlsxwriter
from datetime import date
from datetime import datetime


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')


def face_extractor(img):
    
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
        cropped_face = img[y:y+h, x:x+w]
    
    return cropped_face


video_capture = cv2.VideoCapture(0)
p1 = 'StudentsInfo/'
count=0
row=1
idno=None
workbook = xlsxwriter.Workbook(r'Attendance of '+str(date.today())+'.xlsx')
xlsheet = workbook.add_worksheet()
Columns = ['S. No.' , 'Roll No.', 'Name' , 'Date' , 'Time']
xlsheet.write('A1', Columns[0])
xlsheet.write('B1', Columns[1])
xlsheet.write('C1', Columns[2])
xlsheet.write('D1', Columns[3])
xlsheet.write('E1', Columns[4])
while True:
    _, frame = video_capture.read()
    
    if count == 0:
        cv2.putText(frame,"Show  Me Your Id Card", (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (51, 102, 255), 2)
        idno=decodeTheBarcode(frame)
        if str(idno)=='Stop' or str(idno)=='stop':
            break
        if idno not in StudDetails :
            idno=None
            
    if idno != None :
        count+=1
    
    if count!=0 and idno!= None :  
        cv2.putText(frame,"Look At The Camera", (20,20), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 1)
        face = face_extractor(frame)
        if type(face) is np.ndarray:
            face = cv2.resize(face, (224,224))
            im = Image.fromarray(face, 'RGB')
            img_array = np.array(im)
            img_array = np.expand_dims(img_array, axis = 0)
            name = StudDetails[str(idno)]
            res = DeepFace.verify(img1_path = p1+name+'.jpg'  , img2_path=frame , model_name='VGG-Face' , enforce_detection=False)
            if res['verified']==True :
                Write(idno,row,xlsheet)
                row+=1
                count=0
                cv2.putText(frame, name+" Registered! ", (50,250), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                idno=None
                time.sleep(2)
                
            else:
                cv2.putText(frame, "you are not "+name, (150,130), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
        else:
            cv2.putText(frame, "No Face Found", (100,100), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,0), 1)
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
workbook.close()
video_capture.release()
cv2.destroyAllWindows()
#sendmail()

19501A0552
1/1 [==============================] - 0s 385ms/step
19501A0552
1/1 [==============================] - 0s 340ms/step
19501A0552
1/1 [==============================] - 0s 340ms/step
19501A0552
1/1 [==============================] - 0s 327ms/step
19501A0552
1/1 [==============================] - 0s 332ms/step
19501A0552
1/1 [==============================] - 0s 331ms/step
19501A0552
1/1 [==============================] - 0s 357ms/step
19501A0552
1/1 [==============================] - 0s 330ms/step
19501A0552
1/1 [==============================] - 0s 348ms/step
19501A0552
1/1 [==============================] - 0s 347ms/step
36140507
19501A0552
1/1 [==============================] - 0s 332ms/step
19501A0552
1/1 [==============================] - 0s 342ms/step
19501A0552
1/1 [==============================] - 0s 326ms/step
19501A0552
1/1 [==============================] - 0s 339ms/step
19501A0552
1/1 [==============================] - 0s 359ms/step
19501A0552
1/1 [===============

KeyboardInterrupt: 

## Mailing the data to the provided mail

The data is mailed using smtplib
smtplib is no longer authenticating the gmail with normal password
You need to have the app  specifc password generated by gmail after turning two step verification ON.
<a href="https://stackoverflow.com/questions/73026671/how-do-i-now-since-june-2022-send-an-email-via-gmail-using-a-python-script">know more</a>

In [48]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from datetime import date

def sendmail():
    fromaddr = "myweb.verify@gmail.com"
    toaddr = "sameerfarooq9966@gmail.com"

    msg = MIMEMultipart()

    msg['From'] = fromaddr

    msg['To'] = toaddr

    msg['Subject'] = "list of attendees on "+str(date.today())

    body = '''Hello, I hope your day is going well 
            I'm back with new attendance list,check the below file.
    '''

    msg.attach(MIMEText(body, 'plain'))
    print('Collecting the file....')
    filename = 'Attendance of '+str(date.today())+'.xlsx'
    attachment = open('C:\\Users\\DELL\\'+filename, "rb")

    p = MIMEBase('application', 'octet-stream')

    # To change the payload into encoded form
    p.set_payload((attachment).read())

    # encode into base64
    encoders.encode_base64(p)

    p.add_header('Content-Disposition', "attachment; filename= %s" % filename)

    # attach the instance 'p' to instance 'msg'
    msg.attach(p)

    # creates SMTP session
    s = smtplib.SMTP('smtp.gmail.com', 587)
    print('Session starting....')
    # start TLS for security
    s.starttls()

    # Authentication
    s.login(fromaddr, '***password***')

    # Converts the Multipart msg into a string
    text = msg.as_string()
    print('Sending mail....')
    # sending the mail
    s.sendmail(fromaddr, toaddr, text)
    print('Mail sent successfully!!')
    # terminating the session
    s.quit()


In [9]:
video_capture.release()
cv2.destroyAllWindows()